In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [21]:
train_df = pd.read_csv('./data/trainingset.csv')
test_df = pd.read_csv('./data/testset.csv')
train_df.head()

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,172,255,256,257,258,268,280,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,255,256,257,258,260,268) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cntryid,cntryid_e,age_r,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,...,v85,v50,v89,v127,v239,v224,v71,v105,row,uni
0,Russian Federation,Russian Federation,42.0,Female,Yes,Yes,High,25 to less than 50,25 to less than 50,999,...,Agree,Agree,NaN,NaN,233.0,9996.0,8521,9996.0,95754,jw3580
1,United States,United States,NaN,Male,Yes,Yes,High,75 to less than 90,75 to less than 90,999,...,Agree,Neither agree nor disagree,NaN,NaN,9999.0,9996.0,9999,9996.0,57367,jw3580
2,United States,United States,NaN,Female,Yes,Yes,High,NaN,NaN,999,...,Strongly agree,Strongly agree,NaN,NaN,9996.0,9999.0,9996,9999.0,195554,jw3580
3,Czech Republic,Czech Republic,23.0,Female,Yes,Yes,Medium,10 to less than 25,10 to less than 25,ces,...,Agree,Agree,NaN,NaN,3311.0,9996.0,6419,9996.0,42148,jw3580
4,Canada,Canada (English),NaN,Male,Yes,Yes,Medium,NaN,NaN,999,...,Disagree,Agree,NaN,NaN,9996.0,9999.0,9999,9999.0,86076,jw3580


In [26]:
train_df['gender_r'].value_counts()

Male      12495
Female     7505
Name: gender_r, dtype: int64

In [27]:
test_df['gender_r'].value_counts()

Female    10111
Male       9889
Name: gender_r, dtype: int64

In [3]:
# Dropping uni and v262 because the two columns have no infomration
train_df.drop(['uni', 'v262'], axis=1, inplace=True)
test_df.drop(['uni', 'v262'], axis=1, inplace=True)

# NaN processing with interpolation
train_df.fillna(train_df.mean(), inplace=True)
test_df.fillna(test_df.mean(), inplace=True)

# Pick out categorical columns
train_categ_df = train_df.select_dtypes(include=[np.object], exclude=[np.number])
train_df.drop(train_categ_df.columns, axis=1, inplace=True)
test_categ_df = test_df.select_dtypes(include=[np.object], exclude=[np.number])
test_df.drop(test_categ_df.columns, axis=1, inplace=True)

# One-hot encode
train_categ_df = pd.get_dummies(train_categ_df, dummy_na=True)
test_categ_df = pd.get_dummies(test_categ_df, dummy_na=True)
train_df = pd.concat([train_df, train_categ_df], axis=1)
test_df = pd.concat([test_df, test_categ_df], axis=1)
test_df.columns

# Divide into X and Y (for submission)
X_train_final = train_df.drop('job_performance', axis=1) # Fit on train_final's
y_train_final = train_df['job_performance']
X_test_final = test_df.drop('job_performance', axis=1) # and predict on X_test_final for submission
y_test_final = test_df['job_performance'] # Empty

# Sample Bias Correction

In [48]:
diff_cols1 = set(train_categ_df.columns) - set(test_categ_df.columns)
diff_cols1

{'cnt_brth_Azerbaijan',
 'cnt_brth_Ghana',
 'cnt_brth_Iceland',
 'cnt_brth_Mexico',
 'cnt_brth_Myanmar',
 'cnt_brth_Norway',
 'cnt_brth_Saint Lucia',
 'cnt_brth_The former Yugoslav Republic of Macedonia',
 'cnt_h_Australia',
 'cnt_h_Bangladesh',
 'cnt_h_Brazil',
 'cnt_h_Italy',
 'cnt_h_Kyrgyzstan',
 'cnt_h_Myanmar',
 'cnt_h_Philippines',
 'cnt_h_Republic of Moldova',
 'cnt_h_Senegal',
 'cnt_h_The former Yugoslav Republic of Macedonia',
 'cnt_h_Ukraine',
 'isic2c_39',
 'isic2l_07',
 'isic2l_22.0',
 'isic2l_28.0',
 'isic2l_50.0',
 'isic2l_82.0',
 'isic2l_90.0',
 'isic2l_92.0',
 'isic2l_K',
 'lng_home_997',
 'lng_home_gla',
 'lng_home_isl',
 'lng_home_phi',
 'lng_home_run',
 'lng_home_sin',
 'v109_#50% to less than #75%',
 'v109_Less than #10%',
 'v127_ISCED 4C',
 'v127_ISCED 5A bachelor degree, 5A master degree, and 6 (without distinction)',
 'v172_#75% to less than #90%',
 'v172_#90% or more',
 'v199_#25% to less than #50%',
 'v266_ISCED 4 (without distinction A-B-C)',
 'v266_ISCED 4C',

In [49]:
diff_cols2 = set(test_categ_df.columns) - set(train_categ_df.columns)
diff_cols2

{'cnt_brth_Angola',
 'cnt_brth_Bhutan',
 'cnt_brth_Bulgaria',
 'cnt_brth_Cambodia',
 'cnt_brth_Cape Verde',
 'cnt_brth_Central African Republic',
 'cnt_brth_Chad',
 "cnt_brth_Côte d'Ivoire",
 "cnt_brth_Democratic People's Republic of Korea",
 'cnt_brth_Djibouti',
 'cnt_brth_Ecuador',
 'cnt_brth_El Salvador',
 'cnt_brth_Equatorial Guinea',
 'cnt_brth_French Guiana',
 'cnt_brth_Gibraltar',
 'cnt_brth_Guinea',
 'cnt_brth_Guinea-Bissau',
 'cnt_brth_Guyana',
 'cnt_brth_Honduras',
 'cnt_brth_Iran, Islamic Republic of',
 'cnt_brth_Jamaica',
 'cnt_brth_Jordan',
 'cnt_brth_Kazakhstan',
 "cnt_brth_Lao People's Democratic Republic",
 'cnt_brth_Lebanon',
 'cnt_brth_Libyan Arab Jamahiriya',
 'cnt_brth_Madagascar',
 'cnt_brth_Mali',
 'cnt_brth_Malta',
 'cnt_brth_Mongolia',
 'cnt_brth_Nepal',
 'cnt_brth_New Caledonia',
 'cnt_brth_New Zealand',
 'cnt_brth_Nicaragua',
 'cnt_brth_Panama',
 'cnt_brth_Réunion',
 'cnt_brth_Saint Helena',
 'cnt_brth_Somalia',
 'cnt_brth_Syrian Arab Republic',
 'cnt_brth_Tog

In [50]:
diff_cols = diff_cols1 | diff_cols2

In [51]:
train_categ_values = []
test_categ_values = []
for col in train_categ_df:
    if col not in diff_cols:
        train_categ_values.append(train_categ_df[col].value_counts(normalize=True))
for col in test_categ_df:
    if col not in diff_cols:
        test_categ_values.append(test_categ_df[col].value_counts(normalize=True))
train_categ_values

[0    0.9907
 1    0.0093
 Name: cntryid_Austria, dtype: float64, 0    0.99285
 1    0.00715
 Name: cntryid_Belgium, dtype: float64, 0    0.9363
 1    0.0637
 Name: cntryid_Canada, dtype: float64, 0    0.9828
 1    0.0172
 Name: cntryid_Chile, dtype: float64, 0    0.98055
 1    0.01945
 Name: cntryid_Czech Republic, dtype: float64, 0    0.9933
 1    0.0067
 Name: cntryid_Denmark, dtype: float64, 0    0.99785
 1    0.00215
 Name: cntryid_Estonia, dtype: float64, 0    0.98965
 1    0.01035
 Name: cntryid_Finland, dtype: float64, 0    0.9435
 1    0.0565
 Name: cntryid_France, dtype: float64, 0    0.89695
 1    0.10305
 Name: cntryid_Germany, dtype: float64, 0    0.9896
 1    0.0104
 Name: cntryid_Greece, dtype: float64, 0    0.9944
 1    0.0056
 Name: cntryid_Ireland, dtype: float64, 0    0.9911
 1    0.0089
 Name: cntryid_Israel, dtype: float64, 0    0.9597
 1    0.0403
 Name: cntryid_Italy, dtype: float64, 0    0.91445
 1    0.08555
 Name: cntryid_Japan, dtype: float64, 0    0.94155
 1

In [64]:
diffs = np.abs(np.array(train_categ_values) - np.array(test_categ_values))
diffs

array([0    0.0022
1    0.0022
Name: cntryid_Austria, dtype: float64,
       0    0.0019
1    0.0019
Name: cntryid_Belgium, dtype: float64,
       0    0.0348
1    0.0348
Name: cntryid_Canada, dtype: float64, ...,
       0    0.0031
1    0.0031
Name: v127_ISCED 5A, master degree, and 6 (without distinction), dtype: float64,
       0    0.0003
1    0.0003
Name: v127_ISCED 5B, dtype: float64,
       1    0.0031
0    0.0031
Name: v127_nan, dtype: float64], dtype=object)

In [65]:
diffs_avg = []
for i, diff in enumerate(diffs):
    diffs_avg.append((i, diff.mean()))
diffs_avg

[(0, 0.0021999999999999893),
 (1, 0.0019000000000000063),
 (2, 0.03479999999999998),
 (3, 0.0003999999999999785),
 (4, 0.010000000000000004),
 (5, 0.0022500000000000428),
 (6, 0.0008500000000000087),
 (7, 0.006099999999999997),
 (8, 0.008299999999999988),
 (9, 0.03535),
 (10, 0.0022499999999999873),
 (11, 0.0013000000000000394),
 (12, 0.0021999999999999897),
 (13, 0.008900000000000009),
 (14, 0.017399999999999985),
 (15, 0.013650000000000027),
 (16, 0.0014499999999999756),
 (17, 0.006699999999999964),
 (18, 0.0030499999999999985),
 (19, 0.0004000000000000336),
 (20, 0.004700000000000017),
 (21, 0.05110000000000002),
 (22, 0.004000000000000002),
 (23, 0.004149999999999994),
 (24, 0.0009500000000000034),
 (25, 0.004349999999999982),
 (26, 0.001300000000000039),
 (27, 0.037149999999999954),
 (28, 0.013449999999999983),
 (29, 0.04755000000000004),
 (30, 0.0),
 (31, 0.0021999999999999893),
 (32, 0.031050000000000008),
 (33, 0.0037500000000000155),
 (34, 0.0003999999999999785),
 (35, 0.01000

In [80]:
diffs_avg.sort(key=lambda tup: tup[1],reverse=True)
indices = np.array(diffs_avg[:10])[:,0]
indices

array([1984., 1867., 1954., 1872., 1877., 2028., 2031.,  954., 2006.,
       2012.])

In [81]:
for index in indices:
    print(diffs[int(index)])

0    0.498
1    0.498
Name: v277_Every day, dtype: float64
0    0.4788
1    0.4788
Name: v180_Every day, dtype: float64
0    0.46575
1    0.46575
Name: v233_Every day, dtype: float64
0    0.46405
1    0.46405
Name: v124_At least once a week, dtype: float64
0    0.46405
1    0.46405
Name: v51_At least once a week, dtype: float64
0    0.45395
1    0.45395
Name: v271_nan, dtype: float64
0    0.4535
1    0.4535
Name: v250_nan, dtype: float64
0    0.4534
1    0.4534
Name: ictwork_wle_ca_nan, dtype: float64
0    0.4534
1    0.4534
Name: v195_nan, dtype: float64
0    0.4534
1    0.4534
Name: v73_nan, dtype: float64


For now, it seems that there isn't a very significant difference between train set and test set. Above code finds 50 most biased features after one-hot encoding, but it only finds nan's and survey answers. Gender has around 0.20 difference, which is quite significant, so I do sampling with replacemnt correction for gender only.

# Pipeline for Model Selection and Tuning
Splitting train_df into train and test because only train_df has true labels to check accuracy

In [19]:
from sklearn.model_selection import train_test_split

df = train_df[:]
X = train_df.drop('job_performance', axis=1)
y = train_df['job_performance']
print(X.shape, y.shape)

# SMOTE for reducing sampling bias
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print(X_res.shape, y_res.shape)

(20000, 3329) (20000,)


C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\imblearn\utils\_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got continuous instead.
  TARGET_KIND, type_of_target(y)))


(20000, 3329) (20000,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, y_test.shape

((14000, 3329), (6000,))

In [88]:
def evaluate_MSE(MSE):
    '''Evaluates MSE based on HW5 problem statement'''
    near_upper = 164100
    near_lower = 44900
    unit = 1950
    
    if MSE > near_upper:
        print(f'MSE of {MSE} exceeds the upper bound for 90% range')
    elif MSE > near_lower:
        print(f'MSE of {MSE} is within 90% range for grade')
    else:
        ec = int((near_lower - MSE) / unit)
        print(f'MSE of {MSE} qualifies for {ec}% of EC')

# Create Baseline Model
Basic OLS regression

In [83]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.7416984790578235

In [84]:
y_true = y_test[:]
y_pred = reg.predict(X_test)
y_pred.shape

(6000,)

In [85]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_true, y_pred)
MSE

89451.49947723513

In [89]:
evaluate_MSE(MSE)

MSE of 89451.49947723513 is within 90% range for grade
